<a href="https://colab.research.google.com/github/Rizkyepk/Machine_Learning-Course/blob/main/week_12/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the d2l package using pip
!pip install d2l

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.2
    Uninstalling pyparsing-3.1.2:
      Successfully uninstalled pyparsing-3.1.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: matplotlib-inl

In [2]:
import torch  # Import the torch library for deep learning functionalities
from torch import nn  # Import the neural network module from PyTorch
from torch.nn import functional as F  # Import the functional module from PyTorch for operations like activations
from d2l import torch as d2l  # Import the d2l module for deep learning utilities


In [3]:
class Inception(nn.Module):
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Define the layers for each branch in the Inception module
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        # Forward pass through each branch of the Inception module
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        # Concatenate the outputs of all branches along the channel dimension
        return torch.cat((b1, b2, b3, b4), dim=1)

In [4]:
class GoogleNet(d2l.Classifier):
    def b1(self):
        # Define the layers for the first inception block
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

In [5]:
@d2l.add_to_class(GoogleNet)
def b2(self):
    # Define the layers for the second inception block
    return nn.Sequential(
        nn.LazyConv2d(64, kernel_size=1),
        nn.ReLU(),
        nn.LazyConv2d(192, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

In [6]:
@d2l.add_to_class(GoogleNet)
def b3(self):
    # Define the layers for the third inception block
    return nn.Sequential(
        Inception(64, (96, 128), (16, 32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

In [7]:
@d2l.add_to_class(GoogleNet)
def b4(self):
    # Define the layers for the fourth inception block
    return nn.Sequential(
        Inception(192, (96, 208), (16, 48), 64),
        Inception(160, (112, 224), (24, 64), 64),
        Inception(128, (128, 256), (24, 64), 64),
        Inception(112, (144, 288), (32, 64), 64),
        Inception(256, (160, 320), (32, 128), 128),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

In [8]:
@d2l.add_to_class(GoogleNet)
def b5(self):
    # Define the layers for the fifth inception block
    return nn.Sequential(
        Inception(256, (160, 320), (32, 128), 128),
        Inception(384, (192, 384), (48, 128), 128),
        nn.AdaptiveAvgPool2d((1, 1)),  # Adaptive average pooling to (1, 1)
        nn.Flatten()  # Flatten the output
    )

In [9]:
@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    # Constructor method for the GoogleNet class
    super(GoogleNet, self).__init__()  # Call the constructor of the superclass
    self.save_hyperparameters()  # Save hyperparameters for the model
    # Define the network architecture using sequential layers
    self.net = nn.Sequential(
        self.b1(),  # First inception block
        self.b2(),  # Second inception block
        self.b3(),  # Third inception block
        self.b4(),  # Fourth inception block
        self.b5(),  # Fifth inception block
        nn.LazyLinear(num_classes)  # Output layer
    )
    self.net.apply(d2l.init_cnn)  # Initialize the network paramete

In [10]:
model = GoogleNet().layer_summary((1, 1, 96, 96))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


In [11]:
model = GoogleNet(lr=0.01)  # Instantiate GoogleNet model with learning rate 0.01
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)  # Create a Trainer object for training with 10 epochs and 1 GPU
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))  # Load FashionMNIST dataset with batch size 128 and resize images to 96x96
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)  # Initialize model

100%|██████████| 26421880/26421880 [00:06<00:00, 3799130.26it/s]


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 68267.89it/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:07<00:00, 567019.19it/s] 


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21613890.88it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
